## premier test avec pytorch et premier hook

In [1]:
import requests
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch.nn.functional as F
import torch

In [2]:
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token =  tokenizer.eos_token

In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [4]:
def output_hook(module, input, output):
    print(f'{module} : output')
    print(output)
def input_hook(module, input, output):
    print(f'{module} : input')
    print(input)

def naive_noise_hook(module,input,output): #a rajouter : le fait de selectionner uniquement les bon token, et la bonne variance pour le bruit
    noise = torch.randn_like(output)#*sqrt(3*variance)
    return output+noise

def naive_noise_hook(module,input,output): #a rajouter : le fait de selectionner uniquement les bon token, et la bonne variance pour le bruit
    noise = torch.randn_like(output)#*sqrt(3*variance)
    return output+noise

In [5]:
# hook pour observer ce qu'il se passe dedans
hook1 = model.transformer.wte.register_forward_hook(input_hook)
hook2 = model.transformer.wte.register_forward_hook(output_hook)
hook3 = model.transformer.wpe.register_forward_hook(input_hook)
hook4 = model.transformer.wpe.register_forward_hook(output_hook)
hook5 = model.transformer.drop.register_forward_hook(input_hook)
hook6 = model.transformer.drop.register_forward_hook(output_hook)


In [6]:
#première run sans bruit
input= tokenizer("The Space Needle is in downtown", return_tensors="pt")
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = True, output_attentions =True)
probs = F.softmax(outputs.logits[0, -1, :], dim=-1)
top_probs, top_indices = torch.topk(probs, 4)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

Embedding(50257, 1280) : input
(tensor([[  464,  4687, 10664,   293,   318,   287,  9436]]),)
Embedding(50257, 1280) : output
tensor([[[-0.0092,  0.0426,  0.0258,  ...,  0.0157,  0.0208, -0.0398],
         [-0.0197, -0.0940,  0.0450,  ...,  0.1036, -0.0570,  0.0129],
         [-0.1164,  0.0083, -0.0815,  ...,  0.0397,  0.0729,  0.0875],
         ...,
         [-0.0317, -0.0043, -0.0297,  ...,  0.0258,  0.0306, -0.0274],
         [ 0.0104,  0.0107, -0.0322,  ...,  0.0372, -0.0234, -0.0209],
         [ 0.0529, -0.0317, -0.0876,  ..., -0.0110,  0.0730,  0.0844]]])
Embedding(1024, 1280) : input
(tensor([[0, 1, 2, 3, 4, 5, 6]]),)
Embedding(1024, 1280) : output
tensor([[[ 0.0091,  0.0006,  0.0016,  ..., -0.0340, -0.0054, -0.0056],
         [-0.0045,  0.0030, -0.0066,  ...,  0.0045,  0.0127, -0.0031],
         [-0.0008,  0.0031, -0.0019,  ...,  0.0058, -0.0016,  0.0032],
         ...,
         [ 0.0012, -0.0006,  0.0012,  ...,  0.0066, -0.0016, -0.0075],
         [-0.0010, -0.0002,  0.0048,  

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


 Seattle: 0.8742
 Vancouver: 0.0293
 Portland: 0.0239
 Los: 0.0105


In [7]:
#hook pour ajouter du bruit
noise_hook1 = model.transformer.drop.register_forward_hook(naive_noise_hook)

In [8]:
# deuxième run avec bruit
input= tokenizer("The Space Needle is in downtown", return_tensors="pt")
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = True, output_attentions =True)
probs = F.softmax(outputs.logits[0, -1, :], dim=-1)
top_probs, top_indices = torch.topk(probs, 4)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

Embedding(50257, 1280) : input
(tensor([[  464,  4687, 10664,   293,   318,   287,  9436]]),)
Embedding(50257, 1280) : output
tensor([[[-0.0092,  0.0426,  0.0258,  ...,  0.0157,  0.0208, -0.0398],
         [-0.0197, -0.0940,  0.0450,  ...,  0.1036, -0.0570,  0.0129],
         [-0.1164,  0.0083, -0.0815,  ...,  0.0397,  0.0729,  0.0875],
         ...,
         [-0.0317, -0.0043, -0.0297,  ...,  0.0258,  0.0306, -0.0274],
         [ 0.0104,  0.0107, -0.0322,  ...,  0.0372, -0.0234, -0.0209],
         [ 0.0529, -0.0317, -0.0876,  ..., -0.0110,  0.0730,  0.0844]]])
Embedding(1024, 1280) : input
(tensor([[0, 1, 2, 3, 4, 5, 6]]),)
Embedding(1024, 1280) : output
tensor([[[ 0.0091,  0.0006,  0.0016,  ..., -0.0340, -0.0054, -0.0056],
         [-0.0045,  0.0030, -0.0066,  ...,  0.0045,  0.0127, -0.0031],
         [-0.0008,  0.0031, -0.0019,  ...,  0.0058, -0.0016,  0.0032],
         ...,
         [ 0.0012, -0.0006,  0.0012,  ...,  0.0066, -0.0016, -0.0075],
         [-0.0010, -0.0002,  0.0048,  

In [9]:
hook1.remove()
hook2.remove()
hook3.remove()
hook4.remove()
hook5.remove()
hook6.remove()
noise_hook1.remove()

## dit si un token se réfère au sujet

In [10]:
url = 'https://rome.baulab.info/data/dsets/known_1000.json'
response = requests.get(url) 
data = response.json()

In [11]:
prompts = [dict['prompt'] for dict in data]
subjects = [dict['subject'] for dict in data]
input= tokenizer(prompts, return_tensors="pt", padding= True, return_offsets_mapping= True)

In [12]:
mask = []
for j, prompt in enumerate(prompts):
    map = torch.zeros_like(input.input_ids[j], dtype=torch.int)
    for i,t in enumerate(input.offset_mapping[j]):
        
        if (prompts[j].find(subjects[j])-1<=t[0]) and (t[1]<=prompts[j].find(subjects[j])+len(subjects[j])):
            map[i] = 1
    mask.append(map)
masks_tensor = torch.stack(mask)
masks_tensor = torch.logical_and(masks_tensor, input.attention_mask).int()
masks_tensor

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]], dtype=torch.int32)

pour le prompt i, masks-tensor[i] donne un mask qui dit si oui ou non les tokens se réfèrent au sujet (1 si c'est le cas, 0 sinon)